# 3.5 Hyperparameter Tuning

The choice of model parameters that is optimal for the specific problem can be found automatically by searching the model parameter space. The type of algorithm is fixed in this instance.

There are built-in toolkits to perform the hyperparameter search.

In [1]:
# basic tools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import wget
import h5py
import pandas as pd
import os

In [2]:
import numpy as np
from sklearn.datasets import load_digits,fetch_openml
digits = load_digits()
digits.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])

In [3]:
# explore data type
data,y = digits["data"].copy(),digits["target"].copy()
print(type(data[0][:]),type(y[0]))
# note that we do not modify the raw data that is stored on the digits dictionary.

<class 'numpy.ndarray'> <class 'numpy.int64'>


In [4]:
print(min(data[0]),max(data[0]))
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
scaler = MinMaxScaler()
scaler.fit_transform(data)# fit the model for data normalization
newdata = scaler.transform(data) # transform the data. watch that data was converted to a numpy array

# Split data into 50% train and 50% test subsets
print(f"There are {data.shape[0]} data samples")
X_train, X_test, y_train, y_test = train_test_split(
    data, y, test_size=0.2, shuffle=False)


0.0 15.0
There are 1797 data samples


In [5]:
import sklearn
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Support Vector Machine classifier
clf = SVC(gamma=0.001) # model design
clf.fit(X_train, y_train) # learn
svc_prediction = clf.predict(X_test) # predict on test
print("SVC Accuracy:", metrics.accuracy_score(y_true=y_test ,y_pred=svc_prediction))


SVC Accuracy: 0.9583333333333334


what are the parameters we are trying to optimize?

In [6]:
clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 0.001,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

A search consists of:

* an estimator (regressor or classifier such as ``SVC()``);

* a parameter space;

* a method for searching or sampling candidates;

* a cross-validation scheme; and

* a loss function.


## 1. Grid Search cross validation. 
Performs the search in the brute-force way using cross-validation. One has to define the parameter space. The scikit-learn function is ``GridSearchCV``. More details [here](!https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).





In [7]:
from sklearn.model_selection import GridSearchCV
param_grid = [
  {'C': [1, 5,10,50, 100,500, 1000], 'kernel': ['linear']},
  {'C': [1,5, 10,50, 100,500, 1000], 'gamma': [0.01,0.001, 0.0001], 'kernel': ['rbf']},
 ]

The algorithm will search for all combinations of parameters, which can be from the model algorithms or the choice of features.

In [8]:
search = GridSearchCV(clf, param_grid, cv=5,verbose=3)

In [9]:
search.fit(X_train, y_train) # learn

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV 1/5] END ................C=1, kernel=linear;, score=0.948 total time=   0.0s
[CV 2/5] END ................C=1, kernel=linear;, score=0.948 total time=   0.0s
[CV 3/5] END ................C=1, kernel=linear;, score=0.944 total time=   0.0s
[CV 4/5] END ................C=1, kernel=linear;, score=0.965 total time=   0.0s
[CV 5/5] END ................C=1, kernel=linear;, score=0.972 total time=   0.0s
[CV 1/5] END ................C=5, kernel=linear;, score=0.948 total time=   0.0s
[CV 2/5] END ................C=5, kernel=linear;, score=0.948 total time=   0.0s
[CV 3/5] END ................C=5, kernel=linear;, score=0.944 total time=   0.0s
[CV 4/5] END ................C=5, kernel=linear;, score=0.965 total time=   0.0s
[CV 5/5] END ................C=5, kernel=linear;, score=0.972 total time=   0.0s
[CV 1/5] END ...............C=10, kernel=linear;, score=0.948 total time=   0.0s
[CV 2/5] END ...............C=10, kernel=linear

GridSearchCV(cv=5, estimator=SVC(gamma=0.001),
             param_grid=[{'C': [1, 5, 10, 50, 100, 500, 1000],
                          'kernel': ['linear']},
                         {'C': [1, 5, 10, 50, 100, 500, 1000],
                          'gamma': [0.01, 0.001, 0.0001], 'kernel': ['rbf']}],
             verbose=3)

In [10]:
search.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 0.001,
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(gamma=0.001),
 'n_jobs': None,
 'param_grid': [{'C': [1, 5, 10, 50, 100, 500, 1000], 'kernel': ['linear']},
  {'C': [1, 5, 10, 50, 100, 500, 1000],
   'gamma': [0.01, 0.001, 0.0001],
   'kernel': ['rbf']}],
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 3}

In [11]:
search.cv_results_

{'mean_fit_time': array([0.01232462, 0.0120718 , 0.01202645, 0.01205959, 0.01259809,
        0.01255398, 0.01255393, 0.1116024 , 0.03333983, 0.0278132 ,
        0.11150637, 0.03258362, 0.01712599, 0.10918412, 0.03193731,
        0.01560946, 0.10893884, 0.03155227, 0.01589198, 0.1100811 ,
        0.03147221, 0.0154521 , 0.11023355, 0.03176432, 0.01530852,
        0.10775042, 0.03103819, 0.01555719]),
 'std_fit_time': array([0.00031858, 0.00034351, 0.00032628, 0.00031263, 0.0003713 ,
        0.0003192 , 0.00041668, 0.00199156, 0.00042371, 0.00093848,
        0.00143196, 0.00083066, 0.00050575, 0.00098085, 0.00065373,
        0.00022709, 0.0014512 , 0.00061611, 0.00060015, 0.00133935,
        0.00049222, 0.00035509, 0.00186404, 0.0008341 , 0.00021714,
        0.00150004, 0.00060389, 0.00055426]),
 'mean_score_time': array([0.00367403, 0.0035779 , 0.00359735, 0.00369081, 0.00378022,
        0.00385342, 0.00381751, 0.04939694, 0.02547007, 0.03072257,
        0.04775276, 0.02540679, 0.019693

In [12]:
search.best_params_

{'C': 5, 'gamma': 0.001, 'kernel': 'rbf'}

## 2. Random Search Cross Validation.
It performs the search in the brute-force way using cross-validation. One has to define the parameter space. The scikit-learn function is ``GridSearchCV``. More details [here](!https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

The advantage is that it can be used for a wide hyperparameter space and limit to ``n_iter`` number of iterations.

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform,norm ,loguniform

 
distributions= [ {'C': uniform(loc=1, scale=1000), 'kernel': ['linear']},
  {'C': uniform(loc=1, scale=1000), 'gamma': loguniform(1e-4,1e-2), 'kernel': ['rbf']}]
clf2 = RandomizedSearchCV(clf, distributions, random_state=0,cv=5,n_iter=100)
clf2.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=SVC(gamma=0.001), n_iter=100,
                   param_distributions=[{'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x189622b20>,
                                         'kernel': ['linear']},
                                        {'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x18a71e6d0>,
                                         'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x18a725ee0>,
                                         'kernel': ['rbf']}],
                   random_state=0)

Compare the two

In [14]:
print(search.best_params_)
print(clf2.best_params_)

{'C': 5, 'gamma': 0.001, 'kernel': 'rbf'}
{'C': 20.987665408758737, 'gamma': 0.0007645780792982153, 'kernel': 'rbf'}
